In [ ]:
import string

import pymagnitude
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from scipy.spatial.distance import cosine
import numpy as np

punctuation = string.punctuation + "...—’"

stemmer = SnowballStemmer("english")
stopwords = stopwords.words("english")

In [ ]:
wv_elmo = pymagnitude.Magnitude("/home/nlpsomnath/NegNews/Mueller/embeddings/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.magnitude")
wv_std = pymagnitude.Magnitude("/home/nlpsomnath/NegNews/Mueller/embeddings/crawl-300d-2M.magnitude")

In [ ]:
# s1_raw = "Broker-dealers Citadel Securities, Natixis Securities Americas LLC and MUFG Securities Americas Inc agreed to pay total penalties of more than $6 million to settle charges of providing"
# s2_raw = "Citadel Securities, Natixis & MUFG fined $6 million for blue sheet data failures; Broker-dealers have been fined a combined $6 million for submitting inaccurate blue sheet data to the Securities and Exchange Commission"

s1_raw = "Many Wells Fargo customers woke Thursday to find they couldn’t access their accounts — whether to get cash, pay bills or swipe a debit card for gas. The giant’s online banking…"
s2_raw = "Wells Fargo is facing as much as $1 billion in fines over abuses in the beleaguered bank’s mortgage and auto loan financing divisions, according to a report. ... The prospective penalty comes ..."

In [ ]:
s1 = [word.lower() for word in word_tokenize(s1_raw) if word not in stopwords and word not in punctuation]
print(s1)
print(len(s1))

s2 = [word.lower() for word in word_tokenize(s2_raw) if word not in stopwords and word not in punctuation]
print(s2)
print(len(s2))

In [ ]:
v1_elmo = wv_elmo.query(s1)
v2_elmo = wv_elmo.query(s2)

v1_std = wv_std.query(s1)
v2_std = wv_std.query(s2)

#elmo
sim_elmo = 1 - cosine(np.mean(v1_elmo, axis=0), np.mean(v2_elmo, axis=0))

#non-contextualized
sim_std = 1 - cosine(np.mean(v1, axis=0), np.mean(v2, axis=0))

print(f'elmo: {sim_elmo}; std: {sim_std}')

In [ ]:
search_terms =  [
    'launder',
    'terrorist',
    'counterfeit',
    'unlawful',
    'indict',
    'evasion',
    'embezzle',
    'sanction',
    'felony',
    'extort',
    'smuggle',
    'fraud',
    'violation',
    'convict',
    'guilty',
    'theft',
    'corrupt',
    'bankrupt',
    'suspect',
    'fugitive',
    'arrest',
    'criminal',
    'bribe',
    'accuse',
    'conspire',
    'allege',
    'fined',
    'penalty'
]

search_terms_stemmed = [stemmer.stem(term) for term in search_terms]

In [ ]:
# stem each sentence
s1_stemmed = [stemmer.stem(word) for word in s1]
s2_stemmed = [stemmer.stem(word) for word in s2]

# create vector for each sentence
s1_one_hot = [1 if stemmed_word in search_terms_stemmed else 0 for stemmed_word in s1_stemmed]
s2_one_hot = [1 if stemmed_word in search_terms_stemmed else 0 for stemmed_word in s2_stemmed]

# check to make sure we found at least one search term in each sentence
assert sum(s1_one_hot) > 0, 'no stemmed search terms found in first sentence (s1)'
assert sum(s2_one_hot) > 0, 'no stemmed search terms found in second sentence (s2)'

s1_embedding_weight = [1 / sum(s1_one_hot) if num == 1 else 0 for num in s1_one_hot]
s2_embedding_weight = [1 / sum(s2_one_hot) if num == 1 else 0 for num in s2_one_hot]

v1_std_weighted = np.average(v1_std, axis=0, weights=s1_embedding_weight)
v2_std_weighted = np.average(v2_std, axis=0, weights=s2_embedding_weight)
sim_std_weighted = 1 - cosine(v1_std_weighted, v2_std_weighted)

v1_elmo_weighted = np.average(v1_elmo, axis=0, weights=s1_embedding_weight)
v2_elmo_weighted = np.average(v2_elmo, axis=0, weights=s2_embedding_weight)
sim_elmo_weighted = 1 - cosine(v1_elmo_weighted, v2_elmo_weighted)

print(f'elmo: {sim_elmo_weighted}; std: {sim_std_weighted}')

In [1]:
import string
from collections import defaultdict
import re

from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
import numpy as np

punctuation = string.punctuation + "...—’“”"
stopwords = stopwords.words("english")

def get_max_similarity(wv, sentence_batch1, sentence_batch2, word_tokenizer, stopword_list, punctuation_str, entity_name_list):
    max_similarity = 0
    
    for s1 in sentence_batch1:
        for name in entity_name_list:
            s1 = s1.replace(name, "")
        s1 = re.sub(r" +", " ", s1).strip()
        s1 = [word.lower() for word in word_tokenizer(s1) if word not in stopword_list and word not in punctuation_str]
        for s2 in sentence_batch2:
            for name in entity_name_list:
                s2 = s2.replace(name, "")
            s2 = re.sub(r" +", " ", s2).strip()
            s2 = [word.lower() for word in word_tokenizer(s2) if word not in stopwords and word not in punctuation]
            max_similarity = max(max_similarity, get_sentence_similarity(wv, s1, s2))
    
    return max_similarity

def get_sentence_similarity(wv, s1, s2):
    v1 = np.mean(wv.query(s1), axis=0)
    v2 = np.mean(wv.query(s2), axis=0)
    return 1 - cosine(v1, v2)

def get_semantic_duplicates(wv, date_list, sentence_batch_list, word_tokenizer, stopword_list, punctuation_str, entity_name_list, day_threshold=30, similarity_threshold=0.775):
    dupe_dict = defaultdict(list)
    
    for i in range(0, len(sentence_batch_list)):
        s_i = list(set(sentence_batch_list[i]))
        d_i = date_list[i].date()
        for j in range(i + 1, len(sentence_batch_list)):
            s_j = list(set(sentence_batch_list[j]))
            d_j = date_list[j].date()
            
            if abs((d_i - d_j).days) <= day_threshold:
                max_similarity = get_max_similarity(wv, s_i, s_j, word_tokenizer, stopword_list, punctuation_str, entity_name_list)
                print(f"max similarity between batch {i} and {j} is {max_similarity}")
                if max_similarity >= similarity_threshold:
                    dupe_dict[i].append(j)
    
    return dupe_dict

In [2]:
from datetime import datetime
import pymagnitude

entity_name_list = ["Wells Fargo"]

sentence_batch_list = [
    [
        "Wells Fargo to pay $1B fine for screwing over customers"
        "Wells Fargo, struggling with a host of consumer banking issues since 2016, has once again found itself in the penalty box – this time to the tune of a $1 billion fine from federal regulators.",
    ],
    [
        "Here’s what Wells Fargo did to trigger a $1 billion fine"
        "Wells Fargo’s risk management practices were “reckless, unsafe or unsound.” Unlike many of the scandals that have triggered billion-dollar penalties for banks, the problems that led to a 10 ...",
    ],
    [
        "Wells Fargo hit with record fines, will pay $1 billion for mortgage, auto lending abuses"
        "Today's billion dollar fine is an important development and a fitting penalty given the severity of Wells Fargo's fraudulent and abusive practices, said Pamela Banks, senior policy counsel for ...",
    ],
    [ # unique (3)
        "Colo. To Receive $21.5M in Wells Fargo Fraud Settlement"
        "business Colo. To Receive $21.5M in Wells Fargo Fraud Settlement $575M settlement across the country for banking fraud to be shared by states.",
    ],
    [
        "Wells Fargo facing possible $1 billion in civil penalties over customer-account fraud",
        "“Even though the fine would be enormous by regulator standards, it would not be that large in comparison to Wells Fargo’s overall earnings capability even with recent turmoil,” Seeking Alpha ..."
    ],
    [
        "US to Fine Wells Fargo $1B for Manipulation of Auto, Mortgage Markets | Newsmax.com",
        "Wells Fargo is the nation's largest mortgage lender. This would be the first fine against a bank by the Trump administration since Mick Mulvaney, acting director of the CFPB, took over the bureau in late-November."
    ],
    [
        "Widow's federal lawsuit refiled in wake of $1B penalty",
        "Mississippi woman refiles lawsuit against Wells Fargo in wake of $1B penalty against bank. A Eupora widow's lawsuit against Wells Fargo has been refiled in wake of $1 billion penalty against the ..."
    ],
    [ # unique (7)
        "Wells Fargo Needs Extension To Meet OCC Order",
        "Security & Fraud News: Wells Fargo is gearing up to ask the OCC for an extension to meet a consent order from November of 2015 over anti-money laundering controls. Today’s News Retail"
    ]
]

date_list = [datetime.now()] * len(sentence_batch_list)

wv_std = pymagnitude.Magnitude("/home/nlpsomnath/NegNews/Mueller/embeddings/crawl-300d-2M.magnitude")

In [4]:
from nltk.tokenize import word_tokenize

get_semantic_duplicates(wv_std, date_list, sentence_batch_list, word_tokenize, stopwords, punctuation, entity_name_list)

max similarity between batch 0 and 1 is 0.7989954948425293
max similarity between batch 0 and 2 is 0.8381014466285706
max similarity between batch 0 and 3 is 0.6806859970092773
max similarity between batch 0 and 4 is 0.7451170682907104
max similarity between batch 0 and 5 is 0.7253679037094116
max similarity between batch 0 and 6 is 0.707467257976532
max similarity between batch 0 and 7 is 0.7402647137641907
max similarity between batch 1 and 2 is 0.82061368227005
max similarity between batch 1 and 3 is 0.6600529551506042
max similarity between batch 1 and 4 is 0.734334409236908
max similarity between batch 1 and 5 is 0.6741473078727722
max similarity between batch 1 and 6 is 0.6767853498458862
max similarity between batch 1 and 7 is 0.6779359579086304
max similarity between batch 2 and 3 is 0.7424740791320801
max similarity between batch 2 and 4 is 0.7553139328956604
max similarity between batch 2 and 5 is 0.8081373572349548
max similarity between batch 2 and 6 is 0.7550037503242493
m

defaultdict(list, {0: [1, 2], 1: [2], 2: [5]})

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame()
df['col1'] = ['some sentence here', 'and maybe one here', 'and another there']
df['col2'] = ['next line sentence', 'and another for this line', 'the final sentence']

In [4]:
df['col3'] = list(zip(df['col1'], df['col2']))

In [5]:
df['col4'] = datetime.now()

NameError: name 'datetime' is not defined

In [21]:
get_semantic_duplicates(wv_std, df['col4'], df['col3'], word_tokenize, stopwords, punctuation, entity_name_list)

max similarity between batch 0 and 1 is 0.7482367157936096
max similarity between batch 0 and 2 is 0.7912514209747314
max similarity between batch 1 and 2 is 0.7902313470840454


defaultdict(list, {0: [2], 1: [2]})

In [25]:
df.index[[1, 2]]

Int64Index([1, 2], dtype='int64')

In [8]:
df.loc[['col1', 'col2']].to_dict('records')

[{'col1': 'some sentence here', 'col2': 'next line sentence'},
 {'col1': 'and maybe one here', 'col2': 'and another for this line'},
 {'col1': 'and another there', 'col2': 'the final sentence'}]

In [10]:
col_idx = [0, 2]
df.loc[df.index[[0]], ['col1', 'col2']].to_dict('records')

[{'col1': 'some sentence here', 'col2': 'next line sentence'}]

In [6]:
df

col1                       col2  \
0  some sentence here         next line sentence   
1  and maybe one here  and another for this line   
2   and another there         the final sentence   

                                              col3  
0         (some sentence here, next line sentence)  
1  (and maybe one here, and another for this line)  
2          (and another there, the final sentence)